# Why Neural Network

Neural Network was leveraged as one of the predictive model options for this project, given that:
>
> * The algorithm can recognize hidden patterns and correlations in raw data and cluster, and over time continuously learn and improve
>
> * it provides options and parameters that are adjustable, these parameters can be changed continuously throughout the learning process to improve the overall output result of the model



## Data Preparation and Model Training Process

The following steps were applied to prepare the data and train the model:

1. By Leveraging Alpaca, we extracted trading data for 6 months period, steps taken are as follows:
>
> * Set Alpaca API key and secret key.
>
> * Verify that Alpaca key and secret key were correctly loaded.
>
> * Create the Alpaca API object.
>
> * Reterived the tarding data for the specific ticker for the period required

2. Compile and evaluate a binary classification model using a neural network.

3. Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “signal”. The remaining columns should define the features dataset.

4. Split the features and target sets into training and testing datasets.

5. Use scikit-learn's `StandardScaler` to scale the features data.

### Model Evaluation and Training Process Using a Neural Network

The following steps were completed as part of the evaluation and Training process:

1. Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

2. Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.

3. Utilized small number of epochs when fitting the model

3. Evaluate the model using the test data to determine the model’s loss and accuracy.



In [ ]:
# Initial imports
import os
import requests
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from pathlib import Path
import hvplot.pandas
import holoviews as hv
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [ ]:
load_dotenv()

## Data Retrieval from Alpaca

In [ ]:
# Set Alpaca API key and secret key
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [ ]:
# Verify that Alpaca key and secret were correctly loaded
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

In [ ]:
# Create the Alpaca API object
project2_alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [ ]:
tickers = ["AAPL"]
timeframe = "1Day"
start = pd.Timestamp("2023-01-05", tz="America/New_York").isoformat()
end = pd.Timestamp("2023-08-24", tz="America/New_York").isoformat()

In [ ]:
# Get the trading data for AAPL for 6 months period
p2_portfolio_df = project2_alpaca.get_bars(
    tickers,
    timeframe,
    start = start,
    end = end
).df

# Display sample data
p2_portfolio_df.head()

In [ ]:
hv.extension('bohek')
p2_portfolio_df['close'].hvplot(rot=90)


## Calculate Returns



In [ ]:
# Calculate the daily returns using the closing prices and the pct_change function
p2_portfolio_df['daily_returns'] = p2_portfolio_df['close'].pct_change()

# Calculate the daily returns lagged using the daily returns
p2_portfolio_df['daily_returns_lagged'] = p2_portfolio_df['daily_returns'].shift(1)

# Drop all NaN values from the DataFrame
p2_portfolio_df = p2_portfolio_df.dropna()

# Display sample data
p2_portfolio_df.head()

In [ ]:
p2_portfolio_df[['daily_returns','daily_returns_lagged']].corr().style.background_gradient()

## Define a Simple Momentum Strategy

In [ ]:
# Initialize the new Signal column
p2_portfolio_df['signal'] = 0.0

# When Actual Returns are greater than or equal to 0, generate signal to buy stock long
p2_portfolio_df.loc[(p2_portfolio_df['daily_returns'] >= 0), 'signal'] = 1

# When Actual Returns are less than 0, generate signal to sell stock short
p2_portfolio_df.loc[(p2_portfolio_df['daily_returns'] < 0), 'signal'] = -1

# Drop JPM
p2_portfolio_df = p2_portfolio_df.drop(['symbol'],axis=1)

# Review the DataFrame
display(p2_portfolio_df.head())

In [ ]:
#p2_portfolio_df = p2_portfolio_df.reset_index(drop=True)
#p2_portfolio_df

## Create the features (`X`) and target (`y`) datasets

In [ ]:
# Define the target set y using the signal column
y = p2_portfolio_df['signal']

# Define features set X by selecting all columns but IS_SUCCESSFUL
X = p2_portfolio_df.drop(columns=["signal"])

# Review the features DataFrame
X.head()

# Display a sample of y and X features
display(y.head())
display(X.head())

## Split the features and target sets into training and testing datasets.

In [ ]:
# Split the data into a training and testing dataset and Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Use scikit-learn's `StandardScaler` to scale the features data.

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
len(X_train_scaled[0])


## Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

In [ ]:
# Define the the number of inputs to the model
number_inputs = 9

# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 10

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 5

# Create the Sequential model instance
nn = Sequential()

# Add the first Dense layer specifying the number of inputs, the number of hidden nodes, and the activation function
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_inputs, activation="relu"))

# Add the second Dense layer specifying the number of hidden nodes and the activation function
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(1, activation="sigmoid"))

In [ ]:
# Display the Sequential model summary
nn.summary()

## Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.

In [ ]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
#nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

In [ ]:
# Fit the model using 50 epochs and the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

## Evaluate the model using the test data to determine the model’s loss and accuracy.

In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Save predictions on the test data
predictions = nn.predict(X_test_scaled)
hv.extension('bokeh')
pd.DataFrame(predictions).hvplot()

In [ ]:
# Save predictions on the test data
predictions = nn.predict(X_test_scaled)
predictions

 ### Making Predictions using Dataset

In [ ]:
Sorted_X_test = X_test.sort_index()
X_test
display(Sorted_X_test)

In [ ]:
# assume the prediction > mean is buy, < mean is sell, =mean is do nothing
instr_df = pd.DataFrame()
mean_pred = predictions.mean()
pred_at = 0
for index,row in X_test.iterrows():
    pred = predictions[pred_at]
    pred_at += 1
    if pred > mean_pred:
      signal = 1
    elif pred < mean_pred:
      signal = -1
    else:
      signal = 0

    instr_df = pd.concat([instr_df, pd.DataFrame([[index.date(), 'AAPL', signal]])])
instr_df.sort_values(0, inplace=True)

display(instr_df)

 ### Show result

In [ ]:
from trade import trade_action

In [ ]:
hv.extension('bokeh')
trade_action(instr_df)